# CNN For Identifying Potholes

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from PIL import Image


In [ ]:
# Load and preprocess data
def load_data(json_file, img_dir):
    df = pd.read_json(json_file, lines=True)
    images = []
    masks = []
    for _, row in df.iterrows():
        img_path = os.path.join(img_dir, row['image_path'])
        img = Image.open(img_path).resize((128, 128))  # Resize for simplicity
        img_array = np.array(img) / 255.0  # Normalize
        images.append(img_array)
        
        mask = np.zeros((img.height, img.width))
        for box in row['boxes']:
            x, y, w, h = box['x'], box['y'], box['width'], box['height']
            mask[y:y+h, x:x+w] = 1
        masks.append(mask)
    
    return np.array(images), np.array(masks)


In [ ]:
# Define the CNN model
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Load data
X_train, y_train = load_data('./data/processed/json/ds3_trn.json', './data/processed/')
X_test, y_test = load_data('./data/processed/json/ds3_tst.json', './data/processed/')

# Create and train the model
model = create_model()
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
